In [1]:
%matplotlib inline
import math
import random 
import time
import unittest

from node  import Node
from mcts  import MonteCarloSearchTree
from mcts  import execute_round, default_rollout_policy
from state import AbstractState  as State
from state import AbstractAction as Action
from test  import *

from gomoku_example import *
from maze_unfinished import *
from maze_example    import *

# Mini-Pset: Monte Carlo Tree Search for Collaboration
<img src="img/random-xkcd.png"/>

1. [Part 0: Background Information](#part-0)
  1. [Structure of the Mini-Pset](#structure)
  1. [Breif History](#history)
  1. [MCTS Algorithm](#mcts-alg)
1. [Part 1: Introduction to MCTS](#part-1)
  1. [Provided Classes](#part-1-provided-classes) 
  1. [Implement: `select` (10 points)](#selection)
  1. [Implement: `expand` (10 points)](#expansion)
  1. [Implement: `default_rollout_policy` (10 points)](#rollout-policy)
  1. [Implement: `backpropagate` (10 points)](#backpropagate)
  1. [Written Question: *MCTS with Heuristics* (10 points)](#heuristics)
1. [Part 2: MCTS for Collaboration](#part-2)
  1. [Provided Classes](#part-2-provided-classes) 
  1. [Implement: `Reward`, `is_terminal`, `possible_actions`, `take_action` (10 points each)](#maze)
  1. [Written Question: *Optimal Strategy* (5 points)](#written-optimal)
  1. [Written Question: *Maze Comparison* (5 points)](#written-comparison)

# Part 0: Background Information <a id="part-0"/>

## Structure of the Mini-Pset <a id="structure"/>
This mini-pset is designed to give you an introduction to Monte Carlo Tree Search (MTCS) as a general planning technique as well as an introduction to using MCTS in a collaborative multi-agent scenario. The grading breakdown for this mini-pset can be seen in the outline above.

**Part 0:** provides an overview of the mini-pset, along with helpful background information regarding MCTS. The MCTS algorithm itself is described in the [MCTS Algorithm](#mcts-alg) section. Provided code for completing Part 1 and Part 2 can be found in [part 1](#part-1-provided-classes) and [part 2](#part-2-provided-classes), respectively.

**Part 1:** focuses on introducing the core mechanics of MCTS: *Selection*, *Expansion*, *Simulation*, and *Back-Propagation*.  This is achieved by implementing each of these core methods and then exploring the results of MCTS when applied to [Gomoku](https://en.wikipedia.org/wiki/Gomoku).

**Part 2:** focuses on introducing the key sub-routines that allow the MCTS functions from part 1 to be used in a collaborative multi-agent scenario. This is done by applying MCTS to a multi-agent value collection scenario that is inspired by the Kolumbo mission. 

---
## Breif History <a id="history"/>
TODO -- Finish Description Here

### *"Monte Carlo"* meets *"Tree Search"*
TODO -- Finish Description Here
 <table><tr>
    <td> <img src="img/random-pi-estimate.gif" alt="Drawing" style="width: 300px;"/> </td>
    <td> <img src="img/random-tree-search.gif" alt="Drawing" style="width: 300px;"/> </td>
</tr></table>

### Bandit Problems
<img src="img/random-bandit-cartoon.png" alt="Drawing" style="width: 600px;"/>

*Bandit Problems* are a class of sequential decision making problems where an agent must chose between $K$ actions in order to maximize the cumulative reward of the agent for some amount of time horizon.  For example, the agent (the bandit) in the scenario shown in the image above is an octopus, and the possible actions available to the octopus at any point in time correspond to the available slot machine levers. The goal of the octopus is to devise an optimal policy via experimentation because the underlying reward distribution associated with each lever is unknown. This means that the octopus must estimate the potential reward of pulling a particular lever based on the past observations of taking this action. This introduces the **exploitation-exploration dilemma**: the bandit needs to take advantage of the knowledge it has already gained about the system to make choices associated with high rewards, but the bandit must also must consider unexplored actions to learn more about the underlying reward distribution of said action. 

The best action is not clear because the underlying reward distribution is unknown, and potential rewards must be estimate on past observations. This leads to the exploitation-exploration dilemma. [1] The policy should aim to minimize a player's regret after *n* plays, which is defined as $R_n$ below:

$$R_n = \mu^*n - \mu_j \Sigma_{j=1}^{K} \mathbb{E}[T_k(n)]$$

where $\mu^*$ is the best possible expected reward and $\mathbb{E}[T_j(n)]$ is the expected number of executions of action $j$ across $n$ trials [1]. This leads us to the notion of an Upper Confidence Bound (UCB) that a particular action is optimal because UCB suggests a policy to follow in order to minimize the growth of regret for the bandit over time.

### Upper Confidence Bound (UCB)
TODO -- Finish Description Here

A useful notion for tackling bandit problems is the upper confidence bound (UCB) that a particular action is the optimal action. 

$$UCB1 = \bar{X_j} + \sqrt{\frac{2 \text{ln} (n)}{n_j}}$$ -> first term encourages exploitation (avg reward from arm j), and second encourages exploration (visit less visited choices) [TODO find source for this (see Auer et al)]


---
## MCTS Algorithm <a id="mcts-alg">
TODO -- Finish Description
<img src="img/paper-mcts.png" style="width: 700px;" >

TODO -- Finish Description

two fundamental concepts: the value of an action may be approximated by using random simulation, and that these values may be used efficient to adjust the policy towards a best-first strategy 
 
<img src="img/paper-tree-growth.png" style="width: 400px;" >

### Upper Confidence Bound for Trees (UCT)
TODO -- Finish Description

UCT refers to MCTS with any UCB tree selection policy. We will encode UCT with UCB1 in this problem set.
$$UCT = \bar{X_j} + 2 C_p \sqrt{\frac{2 \text{ln} (n)}{n_j}}$$ (n is how many times the parent node has been visited) -> yields a powerfu form of iterated local search. exploration gaurantees that low-value children are to be chosen eventually. When rewards are in the range [0, 1], Cp optimal = 0.5. 

### Default Policy: Random Rollout 
TODO -- Finish Description

<img src="img/random-tree-rollout.png" style="width: 300px;" >

### Fun Example: AlphaGo Zero
TODO -- Finish Description Here

 <table><tr>
    <td> <img src="img/random-go-performance.gif" alt="Drawing" style="width: 500px;"/> </td>
    <td> <img src="img/random-lee-sedol.jpg" alt="Drawing" style="width: 400px;"/> </td>
</tr></table>

 ---
# Part 1: Introduction to MCTS <a id="part-1"/>

In the first part, you will be asked to finish the MCTS algorithm, where necessary utility functions and classes have been created but the four key steps are still waiting your implementation. After validating your code, you can see how your algorithm helps build a Gomoku AI!

We have provided a Gomoku game which is steill in progress, as shown below. The black is at great advantage, and your job is to help it turn advantage into victory.

<img src="img/gomoku_example_initial_state.png" style="width: 400px;" >

---
## Provided Classes <a id="part-1-provided-classes"/>


You are now asked to finish several key components in Monte Carlo Tree Search, namely the `select`, `expand`, `default_rollout_policy` and `backpropagate` functions. You will need to use the `Node` class we have defined.

### `Node` Class

A `Node` object has the following useful member variables (callable via `my_node.children`, etc):
1. `children`: a dictionary that maps `Action` objects to `Node` objects
1. `tot_reward`: a float of reward that has been back-propagated to this node
1. `num_samples`: an int of number of samples that has been back-propagated to this node

the following properties (callable via `my_node.state`, etc):
1. `state`: a `State` objects corresponds to the node
1. `parent`: a `Node` object that is the parent
1. `unused_edges`: a list of `Action` objects that have not contributed to building child nodes
1. `is_terminal`: a bool representing whether or not the `State` object associated with the node is a terminal state
1. `depth`: an int of the depth of the node. The depth of a root node, which has `None` as its parent, is 1.
1. `is_expanded`: a bool representing whether all possible actions and associated child nodes have been added to the `children` dictionary

and the following methods:
1. `add_child`(`action`): if the action is in the list `unused_edges`, then build a `Node` object `child` and add `action`: `child` to `children`, and returns `child`

When making random selection from a list of variables, use the `random.choice()` function.


---
## Implement: `select` (10 points) <a id="selection"/>
<img src="img/paper-selection.png" style="width: 700px;" >


Selecting which node to expand is based on choosing the node with the maximal upper confidence bound (UCB):
$$UCB=\frac{\text{reward}}{\text{number of samples of the child node}}+\sqrt{2\frac{\ln({\text{number of samples of the parent node})}}{\text{number of samples of the child node}}}$$
where the first term is the average reward per sample and is the exploitation term; the second term, coming from Hoeffding's inequality, will favor less unexplored choices and is the exploration term.

Later when we are choosing the best strategy (not in simulation), we need to choose the action that brings the maximal reward instead of UCB, so a good practice in implementation is to make the second term vanishable:
$$UCB=\frac{\text{reward}}{\text{number of samples of the child node}}+exploration\_const*\sqrt{2\frac{\ln({\text{number of samples of the parent node})}}{\text{number of samples of the child node}}}$$

In [2]:
def select(node: Node, exploration_const: float = 1.0) -> (Action, Node):
    """ Select the best child node based on UCB; if there are multiple
        child nodes with the same max UCB, randomly select one
    :param node: The parent node
    :param exploration_const: The exploration constant in UCB formula
    :return: The action and the corresponding child node it leads to
    """
    ### BEGIN SOLUTION
    best_ucb = -math.inf
    best_actions = []
    for action, child in node.children.items():
        
        for action, child in node.children.items():
            node_ucb = (child.tot_reward / child.num_samples
                        + exploration_const *
                        math.sqrt(2.0 * math.log(node.num_samples) /
                                  child.num_samples))
            if node_ucb > best_ucb:
                best_ucb = node_ucb
                best_actions = [action]
            elif node_ucb == best_ucb:
                best_actions.append(action)
        
    best_action = random.choice(best_actions)
    return best_action, node.children[best_action]
    ### END SOLUTION

In [3]:
test_select(select)
test_ok()

Tests passed!!

---
## Implement: `expand` (10 points) <a id="expansion"/>
<img src="img/paper-expansion.png" style="width: 700px;" >
Suppose you have selected a node and that node is not a terminal node, you need to randomly select an action from the list of available untaken actions, add the associated new child node to the *children* dictionary and return the child node.

Hint: use the functions provided above.


In [4]:
def expand(node: Node) -> Node:
    """ Randomly select an untried action and create a child node based on it
        Return the new child node
    :param node: The parent node
    :return: The child node
    """
    ### BEGIN SOLUTION
    if node.is_expanded:
        raise Exception("Should not expand a node that has already"
                        " been expanded")
        
    # Insert your code here
    action = random.choice(node.unused_edges)
    return node.add_child(action)
    ### BEGIN SOLUTION


In [5]:
test_expand(expand)
test_ok()

Tests passed!!

---
## Implement: `default_rollout_policy` (10 points) <a id="rollout-policy"/>
<img src="img/paper-simulation.png" style="width: 700px;" >


The default rollout policy is to keep randomly selecting an action from the available ones until a terminal state is reached.
A State object has a property is_terminal to represent whether a it is a terminal state, another property possible_actions which is a list of Action objects, and a member method execute_action which takes an Action object as input and returns a copy of a new state.
Hint: use random.choice to randomly select an action from a list of actions.


### Default Rollout Policy


TODO -- Finish Description here

In [6]:
def default_rollout_policy(state: State) -> float:
    """ The default policy for simulation is to randomly (uniform distribution)
        select an action to update the state and repeat the simulation until
        a terminal state is reached
    :param state: The starting state
    :return: The reward at the terminal state
    """
    ### BEGIN SOLUTION
    while not state.is_terminal:
        action = random.choice(state.possible_actions)
        state = state.execute_action(action)
    return state.reward
    ### END SOLUTION

In [7]:
test_default_rollout_policy(default_rollout_policy)
test_ok()

Tests passed!!

---
## Implement: `backpropagate` (10 points) <a id="backpropagate"/>
<img src="img/paper-back-propagation.png" style="width: 700px;" >


After simulation is performed, the next thing is to update the visit counts and rewards for relevant nodes.
Hint: Node objects have a member property parent; older visit count and reward are stored in public member variables num_samples and tot_reward respectively.

In [8]:
def backpropagate(node: Node, reward: float = 0.0) -> None:
    """ Propagate the reward and sample count from the specified node
        back all the way to the root node (the node with None as parent)
    :param node: The node wheresimulation is performed and
        the reward is evaluated
    :param reward: The reward at the node
    """
    ### BEGIN SOLUTION
    while node is not None:
        node.num_samples += 1
        node.tot_reward += reward
        node = node.parent
    ### END SOLUTION



In [9]:
test_backpropagate(expand)
test_ok()

Tests passed!!

---
## Written Question: *MCTS with Heuristics* (10 points) <a id="heuristics"/>

Now you have implemented the MCTS algorithm, you should be ready to use it run it on the Gomoku problem. Note that the execution might take 3-5 minutes. To ensure that the simulation always terminates in a reasonable number of steps and to make sure that the results observed by everyone are similar, we have fixed the random seed.

Does the result look good? Start thinking about why (you do not need to answer it right now).

Which strategy generates a better result? Explain why. Could you come up with some potential methods to improve the performance of MCTS in Gomoku games?

In [10]:
simulate_with_black_sample_arbitrarily(MonteCarloSearchTree)

NameError: name 'expand' is not defined

In [11]:
simulate_with_black_sample_neighborhood(MonteCarloSearchTree)

NameError: name 'expand' is not defined

YOUR ANSWER HERE

---
# Part 2: MCTS for Collaboration <a id="part-2"/>
<img src="img/auv-random-walk.png" style="width: 400px;" >


In part 1, you have implemented the MCTS algorithm; in part 2, you will learn how to model a problem which can call that algorithm

---
## AUV Reward Collection Problem
Suppose you are given 3 AUVs and your task is to use them to explore valuable locations of an underwater area, which has been spatially discretized below:

<img src="img/maze_example_1.png" style="width: 600px;" >

The time is also discretized into 15 steps. At each time step, an AUV can move to one of the 4 neighboring locations as long as it is not an obstacle (the red tiles in the figure). The radius of blue circles represent the information values of different locations: the larger radius corresponds to 3, and the smaller represents 1.

We refer to this example as Maze Example 1. The task is to model this problem so the algorithm developed in Part 1 can be applied (and you will know how we implemented the Gomoku game because they are quite similar).


---
## Provided Classes <a id="part-2-provided-classes"/>

We have provided a class `UnfinishedMazaState`, and your job is to finish building the class `MazeState`. To be more specific, you need to implement 4 key properties/methods, namely `is_terminal`, `reward`, `possible_actions` and `take_action`. You may find the following information useful.

### `MazeAction` Class
1. Can be constructed using `MazeAction(agent_index: int, position: (int, int)`, where `agent_index` indicates which AUV should move, and position is a tuple representing where it is moving to
1. Has properties `agent_index` and `position`

### `MazeEnvironment` Class
1. Has property `rewards`, whcih is a dictionary mapping position (int, int) to reward value (float)
2. Has property `obstacles`, which is a set of position (int, int)

### `UnfinishedMazeState` Class
1. Has a method `is_in_range` that takes (int, int) as input and returns a bool, indicating whether a position is inside the problem domain
1. Has a property `paths` - \[`path_0`, `path_1`, `path_2`, $\ldots$\]. Each `path_i` is a list of position tuples (int, int); the last tuple is current position of agent $i$. This records the trajectories of all agents
1. Has a property `time_remains` which is an int object
1. Has a property `visited` which is a set of visited position tuples (int, int)
1. Has a property `turn` - an int object indicating which agent should move
1. Has a property `switch_agent` which will properly update `turn` so the next agent can move; if all agents have moved for a round, then the `time_remains` property would be updated
1. Has a property `environment`, which is a `MazeEnvironment` object

## Implement: `Reward`, `is_terminal`, `possible_actions`, `take_action` (10 points each) <a id ="maze">

In [12]:
class MazeState(UnfinishedMazeState):

    def __init__(self, environment: MazeEnvironment, time_remains: int = 10):
        """ Create a state of the AUV reward-collection game
        """
        super(UnfinishedMazeState, self).__init__(environment, time_remains)

        
    @property
    def reward(self) -> float:
        """ The total reward at the current state is value of
            rewards that have been visited by agents
        """
        ### BEGIN SOLUTION
        reward = 0.0
        for target_position in self._environment.rewards:
            if target_position in self.visited:
                reward += self._environment.rewards[target_position]
        return reward
        ### END SOLUTION
        
        
    @property
    def is_terminal(self) -> bool:
        """ The only terminal condition is no time remains
        """
        ### BEGIN SOLUTION
        return self.time_remains <= 0
        ### END SOLUTION
    
        
    @property
    def possible_actions(self) -> list:
        """ The possible actions based on the current state
            Note that you are only moving a single agent
        """
        ### BEGIN SOLUTION
        i, j = self._paths[self._turn][-1]
        actions = [MazeAction(self._turn, (i + 1, j)),
                   MazeAction(self._turn, (i - 1, j)),
                   MazeAction(self._turn, (i, j + 1)),
                   MazeAction(self._turn, (i, j - 1))]
        return [MazeAction(self._turn, action.position) for action in actions if
                self.is_in_range(action.position) and
                action.position not in self._environment.obstacles]
        ### END SOLUTION
    
    
    def take_action(self, action: MazeAction) -> "MazeState":
        """ Execute the action based on the current state
            You can assume that the action is always valid
        :param action: The action
        :return: The state after being updated
        """
        ### BEGIN SOLUTION
        self._paths[self._turn].append(action.position)
        self._turn = (self._turn + 1) % len(self._paths)
        if self._turn == 0:  # When all agents have taken a turn of actions
            self._time_remains -= 1
        return self
        ### END SOLUTION

After your code is validated, you can run the simulation. Note that the simulation would take several minutes and we have fixed the random seed.

You can check the check the correctness of your code in incremental steps 
TODO -- mention that tests assume order of implementation is correct

In [13]:
simulate(maze_example_1()).visualize()

TypeError: maze_example_1() missing 1 required positional argument: 'state_class'

In [14]:
test_reward(MazeState)
test_ok()

Tests passed!!

In [15]:
test_is_terminal(MazeState)
test_ok()

Tests passed!!

In [16]:
test_possible_actions(MazeState)
test_ok()

Tests passed!!

In [17]:
test_take_action(MazeState)
test_ok()

Tests passed!!

---
## Written Question: *Optimal Strategy* (5 points) <a id="written-optimal"/>

What is the optimal strategy? You only need to conceptually describe it. There is no need to enter the exact moves for each time steps.

For example:
> The AUV starting at (13, 8) should first collect the reward at (14, 10), (14, 11), and then move towards the reward at (15, 8)
> The AUV starting at (7, 7) should move towards the reward at (11, 2)
> The AUV starting at (12, 12) should move towards the reward at (4, 19)

YOUR ANSWER HERE

Now, run the simulation. The execution will again take several minutes.

In [18]:
simulate(maze_example_1()).visualize()

TypeError: maze_example_1() missing 1 required positional argument: 'state_class'

---
## Written Question: *Maze Comparison* (5 points) <a id="written-comparison"/>


When using the default rollout policy, do you think MCTS will perform equally well in Maze Example 2 as in Maze Example 1? Explain why.

In [19]:
simulate(maze_example_2()).visualize()

TypeError: maze_example_2() missing 1 required positional argument: 'state_class'

YOUR ANSWER HERE

## References 
[1] Browne et. al. 2012, *A Survey of Monte Carlo Tree Search Methods*, IEEE

TODO -- add more references